## Writing Streaming Data to Files

As we have successfully read the data and see it is being processed using `writeStream.format('console')`, now it is time for us to understand how the data can be written to files.

Here are the steps we need to follow to write the data to files:
1. Ensure the logs are being redirected to Netcat Webserver
2. Read the data using `spark.readStream` with `format('socket')`
3. Use `writeStream.format` with appropriate options related to the file format. We will be using `writeStream.format('csv')` and hence we need to specify checkpoint and target location.
```
socketDF \
    .writeStream \
    .format("csv") \
    .option("checkpointLocation", "/FileStore/retail_logs/gen_logs/checkpoint") \
    .option("path", "/FileStore/retail_logs/gen_logs/data") \
    .start()
```
4. Validate both the checkpoint location as well as data location in which files are being copied to.

In [0]:
socketDF = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9000) \
    .load()

In [0]:
%fs rm dbfs:/FileStore/retail_logs/gen_logs -r

res0: Boolean = true

In [0]:
%fs ls dbfs:/FileStore/retail_logs/

path,name,size
dbfs:/FileStore/retail_logs/checkpoint/,checkpoint/,0
dbfs:/FileStore/retail_logs/visitor_traffic/,visitor_traffic/,0


In [0]:
%fs mkdirs dbfs:/FileStore/retail_logs/gen_logs

res2: Boolean = true

In [0]:
%fs mkdirs dbfs:/FileStore/retail_logs/gen_logs/data

res3: Boolean = true

In [0]:
socketDF \
    .writeStream \
    .format("csv") \
    .option("path", "/FileStore/retail_logs/gen_logs/data") \
    .start()

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1406554999831980> in <module> 
 ----> 1 socketDF \ 
 2 . writeStream \ 
 3 . format ( "csv" ) \ 
 4 . option ( "path" , "/FileStore/retail_logs/gen_logs/data" ) \ 
 5 . start ( ) 

 /databricks/spark/python/pyspark/sql/streaming.py in start (self, path, format, outputMode, partitionBy, queryName, **options) 
 1515 self . queryName ( queryName ) 
 1516 if path is None : 
 -> 1517 return self . _sq ( self . _jwrite . start ( ) ) 
 1518 else : 
 1519 return self . _sq ( self . _jwrite . start ( path ) ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : checkpointLocation must be specified either through option("checkpointLocation", ...) or SparkSession.conf.set("spark.sql.streaming.checkpointLocation", ...)

In [0]:
socketDF \
  .writeStream \
  .format("csv") \
  .option("checkpointLocation", "/FileStore/retail_logs/gen_logs/checkpoint") \
  .option("path", "/FileStore/retail_logs/gen_logs/data") \
  .trigger(processingTime='5 seconds') \
  .start()

Out[3]: <pyspark.sql.streaming.StreamingQuery at 0x7f5e11f33ac0>

In [0]:
%fs ls /FileStore/retail_logs/gen_logs/data

path,name,size
dbfs:/FileStore/retail_logs/gen_logs/data/_spark_metadata/,_spark_metadata/,0
dbfs:/FileStore/retail_logs/gen_logs/data/part-00000-1766ac92-de59-4329-9ff5-8b020ea780d7-c000.csv,part-00000-1766ac92-de59-4329-9ff5-8b020ea780d7-c000.csv,0
dbfs:/FileStore/retail_logs/gen_logs/data/part-00000-1bce272e-20b7-4c57-89b3-0fa33b07169e-c000.csv,part-00000-1bce272e-20b7-4c57-89b3-0fa33b07169e-c000.csv,422
dbfs:/FileStore/retail_logs/gen_logs/data/part-00000-2134960e-9053-472e-928b-3384d0eadbc1-c000.csv,part-00000-2134960e-9053-472e-928b-3384d0eadbc1-c000.csv,390
dbfs:/FileStore/retail_logs/gen_logs/data/part-00000-322aa87a-a33b-4727-90e8-eca834913b5c-c000.csv,part-00000-322aa87a-a33b-4727-90e8-eca834913b5c-c000.csv,418
dbfs:/FileStore/retail_logs/gen_logs/data/part-00000-380f9d1f-3d70-47bc-a018-661c4225412d-c000.csv,part-00000-380f9d1f-3d70-47bc-a018-661c4225412d-c000.csv,382
dbfs:/FileStore/retail_logs/gen_logs/data/part-00000-3de4181b-548c-487e-8018-6a4120ae7d36-c000.csv,part-00000-3de4181b-548c-487e-8018-6a4120ae7d36-c000.csv,439
dbfs:/FileStore/retail_logs/gen_logs/data/part-00000-48dbe17c-a925-48fe-9f4e-b26b021254be-c000.csv,part-00000-48dbe17c-a925-48fe-9f4e-b26b021254be-c000.csv,388
dbfs:/FileStore/retail_logs/gen_logs/data/part-00000-60429dab-ec58-43c8-a2f9-0b18a8c9aa61-c000.csv,part-00000-60429dab-ec58-43c8-a2f9-0b18a8c9aa61-c000.csv,359
dbfs:/FileStore/retail_logs/gen_logs/data/part-00000-65469286-fe4d-4836-b4fd-12a09e6003db-c000.csv,part-00000-65469286-fe4d-4836-b4fd-12a09e6003db-c000.csv,392


In [0]:
%fs ls /FileStore/retail_logs/gen_logs/checkpoint

path,name,size
dbfs:/FileStore/retail_logs/gen_logs/checkpoint/commits/,commits/,0
dbfs:/FileStore/retail_logs/gen_logs/checkpoint/metadata,metadata,45
dbfs:/FileStore/retail_logs/gen_logs/checkpoint/offsets/,offsets/,0


In [0]:
%fs head dbfs:/FileStore/retail_logs/gen_logs/data/part-00000-8895135d-18ca-4f84-a2d2-c1db77cc754e-c000.csv

"74.198.176.189 - - [01/Aug/2021:06:20:50 -0800] \"GET /departments HTTP/1.1\" 200 2173 \"-\" \"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:30.0) Gecko/20100101 Firefox/30.0\""
"29.138.96.173 - - [01/Aug/2021:06:20:54 -0800] \"GET /department/team%20sports/categories HTTP/1.1\" 200 2052 \"-\" \"Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0\""